### Imports & Standard Procedure 

In [87]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# RMSE and R^2 
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split as tts 
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, RobustScaler

# XGB regressor
from sklearn.ensemble import GradientBoostingRegressor


# surpress warnings
import warnings
warnings.filterwarnings("ignore", message="No supported index is available", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

pd.set_option('display.max_columns', None)

# Show full width of each column
pd.set_option('display.max_colwidth', None)



### Read data in

In [88]:
df = pd.read_csv("../Stats 170 Market Hotness/Datasets/models_dataset.csv", low_memory=False) 

df.head()

,Unnamed: 0,Year,Month,cntycd,cntyname,med_dom,avg_salelistratio,n,med_buildingareatotal,med_yearbuilt,med_FEDFUNDS,med_propertytax,med_unemploymentrate,med_hpi,med_CPI,med_totalRevenues,med_estimatedPopulation,med_revenuePerCapita,med_interestRate,prior_salelistratio,prior_dom,weight,adj_avg_salelistratio,adj_med_dom,date,scaled_salelistratio,scaled_dom,raw_hotness,hotness_index
0,1,2020,2,6001,ALAMEDA,12.5,1.104670,295,1427.0,1963.0,1.58,779769928,NaN,669.41,274.412,3.752905e+09,1670834.0,2246.0,NaN,1.081750,13.0,0.907692,1.102554,12.546154,2020-02-01,3.924848,0.476011,4.400859,75.477438
1,2,2020,2,6007,BUTTE,14.0,0.955099,199,1800.0,1984.0,1.58,27517338,5.4,669.41,274.412,7.161235e+08,210291.0,3405.0,NaN,0.964700,14.0,0.868996,0.956357,14.000000,2020-02-01,-1.013131,0.353719,-0.659412,42.753370
2,3,2020,2,6013,CONTRA COSTA,14.0,1.027662,255,1780.0,1987.0,1.58,373367933,3.0,669.41,274.412,3.809810e+09,1153561.0,3303.0,NaN,1.020982,13.0,0.894737,1.026959,13.894737,2020-02-01,1.371524,0.362574,1.734097,58.231862
3,4,2020,2,6019,FRESNO,22.0,0.957421,13,1629.0,1977.0,1.58,170548340,NaN,669.41,274.412,2.053548e+09,1023358.0,2007.0,NaN,0.988993,13.0,0.302326,0.979448,15.720930,2020-02-01,-0.233198,0.208961,-0.024237,46.860960
4,5,2020,2,6021,GLENN,36.5,0.964926,16,1552.0,1989.5,1.58,3891615,NaN,669.41,274.412,1.009869e+08,29400.0,3435.0,NaN,0.975615,12.0,0.347826,0.971897,20.521739,2020-02-01,-0.488233,-0.194864,-0.683097,42.600202


In [89]:
# For forecasting on new data 2020-2024 to forecast on 2024
full_df = pd.read_csv('../Stats 170 Market Hotness/Datasets/hotness_index_full.csv') 

# For pure training 
train_df = pd.read_csv('../Stats 170 Market Hotness/Datasets/hotness_index_train.csv')

# For pure testing / validation for forecasting 
test_df = pd.read_csv('../Stats 170 Market Hotness/Datasets/hotness_index_test.csv')

full_df.head()

,Year,Month,cntycd,med_dom,avg_salelistratio,n,med_buildingareatotal,med_yearbuilt,med_FEDFUNDS,med_propertytax,med_unemploymentrate,med_hpi,med_CPI,med_interestRate,prior_salelistratio,prior_dom,weight,adj_avg_salelistratio,adj_med_dom,date,scaled_salelistratio,scaled_dom,raw_hotness,hotness_index
0,2020,2,6001,12.0,1.108364,737,1340.0,1966.0,1.58,779769928,NaN,669.41,274.412,NaN,1.084564,13.0,0.960887,1.107433,12.039113,2020-02-01,3.834077,0.434959,4.269036,89.679479
1,2020,2,6007,13.0,0.960690,504,1663.0,1984.0,1.58,27517338,NaN,669.41,274.412,NaN,0.971680,13.0,0.943820,0.961308,13.000000,2020-02-01,-0.708439,0.368175,-0.340264,74.682891
2,2020,2,6013,14.0,1.021308,697,1712.0,1987.0,1.58,373367933,NaN,669.41,274.412,NaN,1.019923,13.0,0.958735,1.021251,13.958735,2020-02-01,1.154985,0.301541,1.456526,80.528836
3,2020,2,6019,33.0,0.966584,36,1438.0,1975.0,1.58,170548340,NaN,669.41,274.412,NaN,0.991137,13.0,0.545455,0.977745,23.909091,2020-02-01,-0.197469,-0.390033,-0.587502,73.878491
4,2020,2,6021,36.5,0.968134,34,1564.0,1972.5,1.58,3891615,NaN,669.41,274.412,NaN,0.979409,13.0,0.531250,0.973419,25.484375,2020-02-01,-0.331933,-0.499519,-0.831452,73.084787


In [90]:
train_df.head(50)

train_df.shape

(2398, 24)

In [ ]:
# encoder.get_feature_names_out(['cntycd'])


array(['cntycd_6001', 'cntycd_6005', 'cntycd_6007', 'cntycd_6009',
       'cntycd_6011', 'cntycd_6013', 'cntycd_6017', 'cntycd_6019',
       'cntycd_6021', 'cntycd_6025', 'cntycd_6029', 'cntycd_6031',
       'cntycd_6033', 'cntycd_6037', 'cntycd_6039', 'cntycd_6041',
       'cntycd_6043', 'cntycd_6045', 'cntycd_6047', 'cntycd_6051',
       'cntycd_6053', 'cntycd_6055', 'cntycd_6057', 'cntycd_6059',
       'cntycd_6061', 'cntycd_6063', 'cntycd_6065', 'cntycd_6067',
       'cntycd_6069', 'cntycd_6071', 'cntycd_6073', 'cntycd_6075',
       'cntycd_6077', 'cntycd_6079', 'cntycd_6081', 'cntycd_6083',
       'cntycd_6085', 'cntycd_6087', 'cntycd_6089', 'cntycd_6093',
       'cntycd_6095', 'cntycd_6097', 'cntycd_6099', 'cntycd_6101',
       'cntycd_6103', 'cntycd_6105', 'cntycd_6107', 'cntycd_6109',
       'cntycd_6111', 'cntycd_6113', 'cntycd_6115'], dtype=object)

### Setup Data

Gradient boosting with all features: (assuming optimized parameters)

- Gradient Boosting RMSE: 5.1308
- Gradient Boosting R²:   0.7134

with just priors:

- Gradient Boosting RMSE: 5.8750
- Gradient Boosting R²:   0.6242 

In [91]:
features = ['prior_dom',
            'prior_salelistratio',
            'med_yearbuilt', 'med_buildingareatotal',
            'med_FEDFUNDS', 'med_propertytax',  'med_hpi', 'med_CPI', 
            # 'med_unemploymentrate',
            # 'med_totalRevenues',
            #'med_estimatedPopulation', 'med_revenuePerCapita',
            
             ]

med_subset =['med_yearbuilt', 'med_buildingareatotal',
            'med_FEDFUNDS', 'med_propertytax',  'med_hpi', 'med_CPI']

# Set up Train - Test - Split

df_xgb_x = train_df.copy()
df_xgb_y = test_df.copy()




# Ensure 'date' is datetime
df_xgb_x['date'] = pd.to_datetime(df_xgb_x['date'])
df_xgb_y['date'] = pd.to_datetime(df_xgb_y['date'])

# Set date as index (optional)
df_xgb_x.set_index('date', inplace=True)
df_xgb_y.set_index('date', inplace=True)


for col in ['med_unemploymentrate', 'med_hpi', 'med_yearbuilt', 'hotness_index']:
    
    # Try filling with county-level median
    df_xgb_x[col] = df_xgb_x.groupby('cntycd')[col].transform(
        lambda x: x.fillna(x.median())
    )
    
    # Fill any remaining NaNs (where county median was not available) with global median
    df_xgb_x[col] = df_xgb_x[col].fillna(df_xgb_x[col].median())
    
    # Try filling with county-level median
    df_xgb_y[col] = df_xgb_y.groupby('cntycd')[col].transform(
        lambda x: x.fillna(x.median())
    )
    
    # Fill any remaining NaNs (where county median was not available) with global median
    df_xgb_y[col] = df_xgb_y[col].fillna(df_xgb_y[col].median())

# Filter to Feb 2, 2020 – Feb 2, 2025

# No need to filter date anymore as we have separate train and test dataframes. 

# train test split 

train = df_xgb_x
test = df_xgb_y

# Set date as index (optional)

# Endogenous and exogenous variables 
y_train = train['hotness_index']
X_train = train[features]


y_test = test['hotness_index']
X_test = test[features]

In [92]:
print(X_train.isnull().sum())  # Shows which columns have NaNs

print("NaNs in y_test:", np.isnan(y_test).sum())
print("NaNs in preds:", np.isnan(preds).sum())

prior_dom                0
prior_salelistratio      0
med_yearbuilt            0
med_buildingareatotal    0
med_FEDFUNDS             0
med_propertytax          0
med_hpi                  0
med_CPI                  0
dtype: int64
NaNs in y_test: 0
NaNs in preds: 0


### Fit XGB Model on entire dataset

In [93]:
# Fit the Gradient Boosting Regressor
gbr = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.01,
    max_depth=3,
    random_state=42,
    subsample=1
)
gbr.fit(X_train, y_train)

# Predict and evaluate
preds = gbr.predict(X_test)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"Gradient Boosting RMSE: {rmse:.4f}")
print(f"Gradient Boosting R²:   {r2:.4f}")

Gradient Boosting RMSE: 2.1281
Gradient Boosting R²:   0.6936


In [79]:
print(X_train.isnull().sum())  # Shows which columns have NaNs

print("NaNs in y_test:", np.isnan(y_test).sum())
print("NaNs in preds:", np.isnan(preds).sum())


prior_dom                0
prior_salelistratio      0
med_yearbuilt            0
med_buildingareatotal    0
med_FEDFUNDS             0
med_propertytax          0
med_hpi                  0
med_CPI                  0
dtype: int64
NaNs in y_test: 0
NaNs in preds: 0


In [80]:
train_start = train.index.min()
train_end = train.index.max()
test_start = test.index.min()
test_end = test.index.max()

print(f"Train range: {train_start} to {train_end}")
print(f"Test range:  {test_start} to {test_end}")

Train range: 2020-02-01 00:00:00 to 2024-08-01 00:00:00
Test range:  2024-09-01 00:00:00 to 2024-12-01 00:00:00


### Finding optimal Parameters using GridSearchCV

In [94]:
# Grid search over hyperparameters
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0]
}

tscv = TimeSeriesSplit(n_splits=5)

gbr = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(gbr, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate
preds = best_model.predict(X_test)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("Best Params:", grid_search.best_params_)
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

Best Params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1.0}
RMSE: 2.1494
R²: 0.6875
